In [1]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.optimizers.legacy import Adamax, Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.metrics import Mean
from tensorflow.keras.utils import to_categorical
from copy import deepcopy

In [2]:
import import_ipynb

import Model 
from Model import *

import utils
from utils import *

In [3]:
data = pd.read_pickle('mon_data.pkl')

In [4]:
from tensorflow.keras.models import load_model

# 모델 로드
model = load_model('nonbase_model.h5')

In [10]:
# 기존 출력층 제거
model.pop()  # softmax 레이어 제거
model.pop()  # Dense(50) 레이어 제거

# 새로운 출력층 추가 (45개 뉴런)
model.add(Dense(45, activation='softmax', name='new_output'))

# 모델 재컴파일
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [11]:
second_part = split_by_label(data, 50, 94)
train, test = split_train_test(second_part, test_size=0.2, random_state=11)

# 3D ndarray 로 변환 
train_seq, train_label = split_data_label(train)

from sklearn.preprocessing import LabelEncoder

# 레이블을 0~44 범위로 재인덱싱
le = LabelEncoder()
train_label = le.fit_transform(train_label)

train_seq = np.stack(train_seq.values)
train_seq = train_seq[..., np.newaxis]

# train_label = train_label.values
train_label = to_categorical(train_label, num_classes=45)
                

history = model.fit(x=train_seq, y=train_label, epochs=50, verbose=1)
print(f"Data_A training accuracy: {history.history['accuracy'][-1]:.4f}")

Epoch 1/50
225/225 [==============================] - 17s 51ms/step - loss: 2.1619 - accuracy: 0.4526
Epoch 2/50
225/225 [==============================] - 11s 50ms/step - loss: 0.8699 - accuracy: 0.7557
Epoch 3/50
225/225 [==============================] - 11s 51ms/step - loss: 0.5152 - accuracy: 0.8599
Epoch 4/50
225/225 [==============================] - 11s 50ms/step - loss: 0.3834 - accuracy: 0.8944
Epoch 5/50
225/225 [==============================] - 12s 51ms/step - loss: 0.3106 - accuracy: 0.9108
Epoch 6/50
225/225 [==============================] - 12s 52ms/step - loss: 0.2596 - accuracy: 0.9276
Epoch 7/50
225/225 [==============================] - 12s 52ms/step - loss: 0.2285 - accuracy: 0.9354
Epoch 8/50
225/225 [==============================] - 12s 52ms/step - loss: 0.1992 - accuracy: 0.9439
Epoch 9/50
225/225 [==============================] - 12s 52ms/step - loss: 0.1850 - accuracy: 0.9469
Epoch 10/50
225/225 [==============================] - 12s 52ms/step - loss: 0.177

### 95개의 클래스로 model test

In [18]:
# 기존 출력층 제거
model.pop()  # softmax 레이어 제거
model.pop()  # Dense(45) 레이어 제거

# 새로운 출력층 추가
model.add(Dense(95, activation='softmax', name='new_output'))

# 모델 재컴파일
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [19]:
test_seq, test_label = split_data_label(data)
test_seq = np.stack(test_seq.values)
test_seq = test_seq[..., np.newaxis]

test_label = test_label.values
test_label = to_categorical(test_label, num_classes=95)

test = tf.data.Dataset.from_tensor_slices((test_seq, test_label))
test = test.batch(32) #(수정) 모델 자체 배치 존재? - 학습시 fit 디폴트값도 32

final_accuracy = model.evaluate(test)
print(final_accuracy)


594/594 [==============================] - 12s 19ms/step - loss: 4.9399 - accuracy: 0.0036
[4.939947605133057, 0.003631578991189599]
